# [Spin-Locking with waiting]^N times

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizer')
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegenerator')
try:
    pulsedmeasurementlogic
except NameError:
    manager.startModule('gui', 'pulsedmeasurementlogic')   
try:
    poimanager
except NameError:
    manager.startModule('logic', 'poimaganer')  
    
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurementlogic.savelogic().get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [7]:
# for single frequency XY8-N:
HHSW = OrderedDict()
HHSW['tau_start'] = 10.0*1e-9
HHSW['tau_incr'] = 10.0*1e-6
HHSW['points'] = 20
HHSW['sweeps']=1000000

In [8]:
tau = HHSW['tau_start'] + np.arange(HHSW['points']) * HHSW['tau_incr']
poi=poimanager.active_poi.get_key()
NV_name =poimanager.poi_list[poi]._name

start_time = time.time()
for t in tau:
    generate_sequence(t)
    time.sleep(4)
    refocus_poi(poi)
    time.sleep(5)
        
    # If everything is properly set, we can start a measurement simply by calling:
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    # Wait until the pulsedmeasurementlogic is actually busy and the measurement is running
    while pulsedmeasurementlogic.module_state() != 'locked':
        time.sleep(0.2)
        
    user_terminated = False
    while fastcounter.get_sweeps() < HHSW['sweeps']:
        if pulsedmeasurementlogic.module_state()=='idle':
            user_terminated = True
            time.sleep(0.2)    
            break
        time.sleep(0.5)
        
    pulsedmasterlogic.toggle_pulsed_measurement(False)
    # Wait until the pulsedmeasurementlogic is actually idle and the measurement is stopped
    while pulsedmeasurementlogic.module_state() == 'locked':
        time.sleep(0.2)
    
    pulsedmasterlogic.save_measurement_data(tag='HHwait_'+f"{t:.{1}e}"+'_'+NV_name, with_error=True)        
    print((time.time() - start_time)/60)    

131233.83333333334 1.8325460541353078e-08
2.2195107380549115
131233.83333333334 1.316229216426102e-08
4.785993583997091
131233.83333333334 1.3680150481375351e-08
7.684195427099864
131233.83333333334 9.613445533680144e-09
10.896059234937033
131233.83333333334 2.128730272682119e-09
14.44655814965566
131233.83333333334 2.4342374904547014e-09
18.331438132127126
131233.83333333334 6.493919955789783e-09
22.55091909567515
131233.83333333334 8.091574577094671e-09
27.094040822982787
131233.83333333334 7.129114013657401e-09
31.97450540860494
131233.83333333334 8.1142388408413e-09
37.20929340521494
131233.83333333334 1.1280628529580914e-08
42.757800511519115
131233.83333333334 5.14685418265343e-09
48.64084175825119
131233.83333333334 1.79501235632372e-10
54.856978034973146
131233.83333333334 1.3199883726003624e-08
61.40379220247269
131233.83333333334 4.998085088832256e-09
68.30054816007615
131233.83333333334 7.641370128004123e-09
75.52947613398234
131233.83333333334 1.375469138668986e-09
83.08037

In [34]:
f"{t:.{1}e}"

'2.0e-05'

In [4]:
def generate_sequence(tau):

    # Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
    # generate_params = pulsedmasterlogic.generate_method_params['rabi']
    generate_params=OrderedDict()
    generate_params['name'] = 'HHwait'
    generate_params['spinlock_amp'] = 0.1112
    generate_params['lock_time'] = 42.8e-6
    generate_params['idle_time'] = tau
    generate_params['alternating'] = True

    # Call the actual generation method using a wrapper method and pass the parameter set
    pulsedmasterlogic.generate_predefined_sequence('Hartmann_Hahn_waiting', generate_params)
    # Give it a moment to generate
    time.sleep(0.5)

    # The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
    # You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
    #for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
    #    print(ensemble_name)

    # The "with_load" flag can be used to also load the waveform into the channels after creating it
    pulsedmasterlogic.sample_sequence('HHwait', with_load=True)
    # Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
    while pulsedmasterlogic.status_dict['sampload_busy']:
        time.sleep(0.1)

    pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                               controlled_variable= [tau],
                                               number_of_lasers=2, 
                                               laser_ignore_list=[], 
                                               alternating=True, 
                                               units=('s', 'arb. u.'))
    time.sleep(0.5)

    pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=1)
    time.sleep(0.5)

    import winsound
    frequency = 300  # Set Frequency To 2500 Hertz
    duration = 1000  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    # You can get a read-only view of the current measurement settings by using this property:
    # for setting, value in pulsedmasterlogic.measurement_settings.items():
    #     print('{0}:\n  {1}'.format(setting, value))

In [2]:
def refocus_poi(poi=None, max_drift=0.5, min_counts=10e+3):
    if poi==None:
        return False
    scannerlogic.stop_scanning()
    poimanager.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    NV_name =poimanager.poi_list[poi]._name
    poimanager.optimise_poi(poi)
    while optimizer.module_state() == 'idle':
        time.sleep(0.2)
    while optimizer.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanager.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= NV_name, 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= NV_name, 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True